# THIS NOTEBOOK IS DEDICATED TO THE ANALYSIS OF 

# *CoT task*


In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
from IPython.display import clear_output, display, HTML, Image
import matplotlib.cm as cm
import warnings
import types
import inspect
warnings.filterwarnings("ignore")
import sys, time
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import mlab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from pylab import *
import matplotlib.backends.backend_pdf
from sklearn.decomposition import KernelPCA

ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
CWD=os.getcwd()
os.chdir(CommonNoteBookesPath)
root='/data'
%run UtilityTools.ipynb
%run Animal_Tags.ipynb
%run Lesion_Size.ipynb
%run BatchRatBehavior.ipynb
%run plotRat_documentation_3_KinematicsInvestigation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run loadRat_documentation.ipynb
%run RunBatchRat_3_CompareGroups.ipynb
os.chdir(CWD)

# logging.getLogger().setLevel(logging.WARNING)
defaultParam={
            "treadmillRange":[0,65],
            "binSize":0.25,
            "trialOffset":20., #max end of trial, in seconds (position will be cutted)
            "sigmaSmoothPosition":0.1,  #smooth the position
            #"sigmaSmoothPosition":0.33 for pavel dataType
            "sigmaSmoothSpeed":0.3, #smooth the speed
            "positionDiffRange": [2.,5.], #min and max differences allowed between two consecutive positions
                                          #min to correct start, max to correct jumps
            "pawFrequencyRange":[2.,10.],
            "startAnalysisParams":[10,0.2,0.5],
            "cameraToTreadmillDelay":2., #seconds, usual time between camera start and treadmill start
            "nbJumpMax" : 100., #if jumps>nbJumpMax, trial is badly tracked


            #parameter to detect end of trial (first position minima)
            "endTrial_backPos":55,  # minima is after the animal went once to the back (after first time position>backPos)
            "endTrial_frontPos":30, # minima's position is in front of treadmill (position[end]<frontPos)
            "endTrial_minTimeSec":4, # minima is after minTimeSec seconds (time[end]>minTimeSec)
    }

print('Done')

# DEFINITIONS

In [ ]:
profile={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':['0','10','8'],
         'Speed':'0',
         'Tag':['CoT-Late_DLS','CoT']
         }

animalList=['Rat369','Rat371','Rat365','Rat367']

animalList=['Rat367','Rat368']

In [ ]:
sessionList=batch_get_session_list(root,animalList,profile)['Sessions']
sessionList

In [ ]:
data.position[18].max()

In [ ]:
speed=[]
for session in sessionList:
    data=Data(root,session[:6],session,saveAsPickle=False,param=defaultParam)
    spd=forwardRunningSpeed(data).compute().values()
    speed.extend(list(spd))
    print(session[:6],np.nanmedian(list(spd)),np.nanstd(list(spd)))
print(np.nanmedian(speed),np.nanstd(speed))    

In [ ]:
for pos in get_positions_array_end_StereotypedTrials(data):
    plt.plot(pos)
plt.show()

In [ ]:
a=get_positions_array_end_StereotypedTrials(data)

In [ ]:
speed=forwardRunningSpeed(data).compute()

In [ ]:
np.nanmedian(list(speed.values())),np.nanstd(list(speed.values()))